In [149]:
import numpy as np
import pandas as pd
import math
import sys

- Default value:

In [150]:
MAX_ELEMENT = 200
SIZE_ELEMENT = 23
MAX_DEPTH = 1000
EXPONENT = 2

sys.setrecursionlimit(10000)

- Read data:

In [151]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

X_train = train.iloc[:, :-1].to_numpy()
y_train = train.iloc[:, -1].to_numpy()

X_test = test.iloc[:, :-1].to_numpy()
y_test = test.iloc[:, -1].to_numpy()

- Fitness function:

In [152]:
def find_weight(X, y):
    return np.linalg.inv(X.T @ X) @ X.T @ y

def calc_y_predict(X, w):
    return np.sum(X * w.T, axis = 1)

def MAE(y_predict, y):
    return np.mean(np.abs(y_predict - y))

def fitness_function(ind):
    w = find_weight(X_train[:, ind] ** EXPONENT, y_train)
    y_predict = calc_y_predict(X_test[:, ind] ** EXPONENT, w)
    return MAE(y_predict, y_test) 

- Generate sample:

In [153]:
sample = []

for _ in range(MAX_ELEMENT):
    sample.append(np.random.randint(2, size=SIZE_ELEMENT).astype(bool))

sample.sort(key = fitness_function)

- Genetic Algorithm:

In [154]:
def GeneticAlgorithm(sample, index = 0):
    if (index == MAX_DEPTH):
        return
    
    if (index % 100 == 0):
        print("Generation: ", index)
        
    next_generation = []

    # 10% of the population are the elite
    for i in range(0, math.ceil(0.1 * MAX_ELEMENT)):
        next_generation.append(sample[i])

    # Create 90% of the population by crossover
    # 50% of the best population are the parents
    for i in range(0, math.ceil(0.9 * MAX_ELEMENT)):
        parent1 = sample[np.random.randint(0, 0.5 * MAX_ELEMENT)]
        parent2 = sample[np.random.randint(0, 0.5 * MAX_ELEMENT)]

        # Crossover
        child = np.zeros_like(parent1)

        for x in range(0, len(child)):
            rate = np.random.rand()

            if (rate < 0.45):
                child[x] = parent1[x]
            elif (rate < 0.9):
                child[x] = parent2[x]
            else:
                child[x] = np.random.randint(0, 2)

        # Add child to the next generation
        next_generation.append(child)

    sample = sorted(sample[:MAX_ELEMENT], key = fitness_function)
    
    GeneticAlgorithm(next_generation, index + 1)

In [155]:
GeneticAlgorithm(sample)
print(sample[0])
print(fitness_function(sample[0]))

Generation:  0
Generation:  100
Generation:  200
Generation:  300
Generation:  400
Generation:  500
Generation:  600
Generation:  700
Generation:  800
Generation:  900
[False  True  True  True False False  True False False  True  True  True
 False  True  True  True False False  True  True  True  True False]
99767.23780796444
